In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 33.7 MB/s 
     |████████████████████████████████| 163 kB 70.3 MB/s 


In [ ]:
pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 798 kB 4.7 MB/s 
     |████████████████████████████████| 529 kB 55.4 MB/s 
     |████████████████████████████████| 87 kB 6.8 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115940 sha256=7b33c2a0acf600a23f23fddee440c5d5a1d68fdb852ca89b7283226c9654efe5
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [ ]:
# import packages
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import tokenizers
from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, AutoConfig, get_linear_schedule_with_warmup

import pytorch_lightning as pl
from torchmetrics.functional import accuracy, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 1

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)

INFO:lightning_lite.utilities.seed:Global seed set to 1


1

In [ ]:
# connect to google drive
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning"

os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['lightning_logs',
 'checkpoints',
 'answer_t4.csv',
 'answer_t5.csv',
 'answer_t6.csv',
 'answer_t7.csv',
 'answer_t8.csv',
 'answer_t10.csv',
 'bert_train_text.csv',
 'vocab.txt',
 'pretrained+bert+pytrochlightning.ipynb']

In [ ]:
# import train and test data
df = pd.read_csv('./data/train_final.csv')
df.drop('Unnamed: 0', axis =1, inplace = True)
df['context'] = df['name'] + '. ' + df['document_text']

test_df = pd.read_csv('./data/test.csv')
test_df['context'] = test_df['name'] + '. ' + test_df['document_text']

In [ ]:
# generate target columns
label_cols = [col for col in df.columns if col not in ['id', 'name', 'document_text', 'context']]
label_cols

['Accounting and Finance',
 'Antitrust',
 'Banking',
 'Broker Dealer',
 'Commodities Trading',
 'Compliance Management',
 'Consumer protection',
 'Contract Provisions',
 'Corporate Communications',
 'Corporate Governance',
 'Definitions',
 'Delivery',
 'Examinations',
 'Exemptions',
 'Fees and Charges',
 'Financial Accounting',
 'Financial Crime',
 'Forms',
 'Fraud',
 'IT Risk',
 'Information Filing',
 'Insurance',
 'Legal',
 'Legal Proceedings',
 'Licensing',
 'Licensure and certification',
 'Liquidity Risk',
 'Listing',
 'Market Abuse',
 'Market Risk',
 'Monetary and Economic Policy',
 'Money Services',
 'Money-Laundering and Terrorist Financing',
 'Natural Disasters',
 'Payments and Settlements',
 'Powers and Duties',
 'Quotation',
 'Records Maintenance',
 'Regulatory Actions',
 'Regulatory Reporting',
 'Required Disclosures',
 'Research',
 'Risk Management',
 'Securities Clearing',
 'Securities Issuing',
 'Securities Management',
 'Securities Sales',
 'Securities Settlement',
 'Tra

In [ ]:
train_df, val_df = train_test_split(df, train_size = 0.8,test_size = 0.2, random_state=1)

In [ ]:
text_file = df['context'].values.tolist() + test_df['context'].values.tolist()

In [ ]:
pd.Series(text_file).to_csv('bert_train_text.csv', index=False, header=False)

In [ ]:
bwpt = tokenizers.BertWordPieceTokenizer()
bwpt.train(
    files = ['bert_train_text.csv'],
    vocab_size=35000, # 这里预设定的词语大小不是很重要
    min_frequency=1,
    limit_alphabet=1000
)
bwpt.save_model('./')

['./vocab.txt']

In [ ]:
bert_model_name = 'bert-base-cased' # cased
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
tokenizer = BertTokenizer(vocab_file='./vocab.txt')

In [ ]:
class RegulationDataset(Dataset):

    def __init__(self, data: pd.DataFrame, tokenizer: BertTokenizer, max_token_len: int = 256):

        self.data = data
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):

        data_row = self.data.iloc[index]

        context = data_row.context
        labels = data_row[label_cols]

        encoding = self.tokenizer.encode_plus(
            context,
            add_special_tokens=True,
            max_length=self.max_token_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return dict(
            context=context,
            input_ids=encoding['input_ids'].flatten(),
            attention_mask=encoding['attention_mask'].flatten(),
            labels=torch.FloatTensor(labels)
        )

In [ ]:
class RegulationDatasetModule(pl.LightningDataModule):

    def __init__(self, train_df, test_df, tokenizer, train_batch_size, test_batch_size, max_token_len=512):
        
        super().__init__()
        self.train_batch_size = train_batch_size
        self.test_batch_size = test_batch_size
        self.train_df = train_df
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        self.train_dataset = RegulationDataset(
            self.train_df,
            self.tokenizer,
            self.max_token_len
        )

        self.test_dataset = RegulationDataset(
            self.test_df,
            self.tokenizer,
            self.max_token_len
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.train_batch_size,
            shuffle=True,
            num_workers=0
        )
    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.test_batch_size,
            num_workers=0
        )
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.test_batch_size,
            num_workers=0
        )

In [ ]:
n_epochs = 20
train_batch_size = 4
test_batch_size = 64
max_token_len = 384

data_module = RegulationDatasetModule(train_df, val_df, tokenizer, train_batch_size, test_batch_size ,max_token_len)
data_module.setup()

In [ ]:
configuration = AutoConfig.from_pretrained(bert_model_name, return_dict=True)
configuration.hidden_dropout_prob = 0.3
configuration.attention_probs_dropout_prob = 0.3

In [ ]:
class RegulationTagger(pl.LightningModule):
    
    def __init__(self, n_classes: int, dropout_ratio, n_training_steps=None, n_warmup_steps=None, lr=0.0005):
        super().__init__()

        self.bert = BertModel.from_pretrained(bert_model_name, return_dict=True)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_ratio),
            nn.Linear(self.bert.config.hidden_size, n_classes)
        )
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.BCELoss()
        self.lr = lr

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = self.classifier(output.pooler_output)
        output = torch.sigmoid(output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
        return loss, output

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": outputs, "labels": labels}

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": outputs, "labels": labels}

    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def training_epoch_end(self, outputs):

        labels = []
        predictions = []

        for output in outputs:
            for out_labels in output["labels"].detach().cpu():
                labels.append(out_labels)

            for out_predictions in output["predictions"].detach().cpu():
                predictions.append(out_predictions)

        labels = torch.stack(labels)
        predictions = torch.stack(predictions)

        for i, name in enumerate(label_cols):
            class_roc_auc = auroc(predictions[:, i], labels[:, i].long())
            self.logger.experiment.add_scalar(f"{name}_roc_auc/Train", class_roc_auc, self.current_epoch)
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.lr)

        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.n_warmup_steps,
            num_training_steps=self.n_training_steps
        )

        return dict(
            optimizer=optimizer,
            lr_scheduler=dict(
                scheduler=scheduler,
                interval='step'
                )
            )

In [ ]:
steps_per_epoch=len(train_df) // train_batch_size
total_training_steps = steps_per_epoch * n_epochs

warmup_steps = total_training_steps // 3

In [ ]:
RegulationTagger(
    n_classes=50,
    dropout_ratio=0,
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps,
    lr=0.0001
)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RegulationTagger(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [ ]:
model = RegulationTagger(
    n_classes=50,
    dropout_ratio=0,
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps,
    lr=0.0001
)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

In [ ]:
logger = TensorBoardLogger("lightning_logs", name="regulation_context")

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
unfreeze_layers = ['layer.10','layer.11','bert.pooler', 'classifier.weight', 'classifier.bias']
#'layer.10','layer.11',
for name ,param in model.named_parameters():
    param.requires_grad = False
    for ele in unfreeze_layers:
        if ele in name:
            param.requires_grad = True
            break


In [ ]:
trainer = pl.Trainer(
    logger=logger,
    callbacks=[checkpoint_callback,early_stopping_callback],
    max_epochs=60,
    gpus=1
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
CUDA_LAUNCH_BLOCKING=1

In [ ]:
trainer.fit(model, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:606: UserWarning: Checkpoint directory /content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


In [ ]:
trainer.fit(model, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:606: UserWarning: Checkpoint directory /content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type       | Params
------------------------------------------
0 | bert       | BertModel  | 108 M 
1 | classifier | Sequential | 38.5 K
2 | criterion  | BCELoss    | 0     
------------------------------------------
108 M     Trainable 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 12, global step 12818: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 13, global step 13804: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 14, global step 14790: 'val_loss' was not in top 1


In [ ]:
train_df, val_df = train_test_split(df, train_size = 0.9,test_size = 0.1, random_state=42)

In [ ]:
n_epochs = 40
train_batch_size = 16
test_batch_size = 64
max_token_len = 384

data_module = RegulationDatasetModule(train_df, val_df, tokenizer, train_batch_size, test_batch_size ,max_token_len)
data_module.setup()

In [ ]:
steps_per_epoch=len(train_df) // train_batch_size
total_training_steps = steps_per_epoch * n_epochs

warmup_steps = total_training_steps // 3

In [ ]:
continuetrainer = pl.Trainer(max_epochs=40,
                             gpus=1,
                             logger=logger,
                             callbacks=[checkpoint_callback,early_stopping_callback],
                             resume_from_checkpoint="/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt")


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:54: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v2.0. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  "Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU 

In [ ]:
best_model = RegulationTagger.load_from_checkpoint(
    '/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt',
    n_classes=50,
    dropout_ratio=0.5,
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps,
    lr=0.00002
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
for name ,param in best_model.named_parameters():
    param.requires_grad = True

In [ ]:
unfreeze_layers = ['layer.10','layer.11','bert.pooler', 'classifier.weight', 'classifier.bias']#'layer.10','layer.11',
for name ,param in best_model.named_parameters():
    param.requires_grad = False
    for ele in unfreeze_layers:
        if ele in name:
            param.requires_grad = True
            break


In [ ]:
continuetrainer.fit(best_model, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1871: LightningDeprecationWarning: `trainer.resume_from_checkpoint` is deprecated in v1.5 and will be removed in v2.0. Specify the fit checkpoint path with `trainer.fit(ckpt_path=)` instead.
  stacklevel=5,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:606: UserWarning: Checkpoint directory /content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will

Sanity Checking: 0it [00:00, ?it/s]

Training: 493it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 14, global step 12880: 'val_loss' reached 0.10823 (best 0.10823), saving model to '/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 15, global step 13373: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 16, global step 13866: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 17, global step 14359: 'val_loss' was not in top 1


In [ ]:
n_epochs = 40
train_batch_size = 16
test_batch_size = 64
max_token_len = 384

data_module = RegulationDatasetModule(train_df, val_df, tokenizer, train_batch_size, test_batch_size ,max_token_len)
data_module.setup()

In [ ]:
steps_per_epoch=len(train_df) // train_batch_size
total_training_steps = steps_per_epoch * n_epochs

warmup_steps = total_training_steps // 3

In [ ]:
continuetrainer1 = pl.Trainer(max_epochs=40,
                             gpus=1,
                             logger=logger,
                             callbacks=[checkpoint_callback,early_stopping_callback],
                             resume_from_checkpoint="/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt")

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:54: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v2.0. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  "Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU 

In [ ]:
best_model1 = RegulationTagger.load_from_checkpoint(
    '/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt',
    n_classes=50,
    dropout_ratio=0.5,
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps,
    lr=0.0001
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
unfreeze_layers = ['layer.1','layer.2','bert.pooler', 'classifier.weight', 'classifier.bias']#'layer.10','layer.11',
for name ,param in best_model1.named_parameters():
    param.requires_grad = False
    for ele in unfreeze_layers:
        if ele in name:
            param.requires_grad = True
            break


In [ ]:
continuetrainer1.fit(best_model1, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1871: LightningDeprecationWarning: `trainer.resume_from_checkpoint` is deprecated in v1.5 and will be removed in v2.0. Specify the fit checkpoint path with `trainer.fit(ckpt_path=)` instead.
  stacklevel=5,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:606: UserWarning: Checkpoint directory /content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will

Sanity Checking: 0it [00:00, ?it/s]

Training: 247it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 16, global step 13620: 'val_loss' reached 0.10780 (best 0.10780), saving model to '/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 18, global step 14606: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 19, global step 15099: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 20, global step 15592: 'val_loss' was not in top 1


In [ ]:
n_epochs = 40
train_batch_size = 16
test_batch_size = 64
max_token_len = 384

data_module = RegulationDatasetModule(train_df, val_df, tokenizer, train_batch_size, test_batch_size ,max_token_len)
data_module.setup()

In [ ]:
steps_per_epoch=len(train_df) // train_batch_size
total_training_steps = steps_per_epoch * n_epochs

warmup_steps = total_training_steps // 3

In [ ]:
continuetrainer2 = pl.Trainer(max_epochs=40,
                             gpus=1,
                             logger=logger,
                             callbacks=[checkpoint_callback,early_stopping_callback],
                             resume_from_checkpoint="/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt")

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:54: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v2.0. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  "Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU 

In [ ]:
best_model2 = RegulationTagger.load_from_checkpoint(
    '/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt',
    n_classes=50,
    dropout_ratio=0.5,
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps,
    lr=0.00005
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
unfreeze_layers = ['layer.3','layer.4','layer.5','bert.pooler', 'classifier.weight', 'classifier.bias']#'layer.10','layer.11',
for name ,param in best_model2.named_parameters():
    param.requires_grad = False
    for ele in unfreeze_layers:
        if ele in name:
            param.requires_grad = True
            break

In [ ]:
continuetrainer2.fit(best_model2, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:606: UserWarning: Checkpoint directory /content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type       | Params
------------------------------------------
0 | bert       | BertModel  | 108 M 
1 | classifier | Sequential | 38.5 K
2 | criterion  | BCELoss    | 0     
------------------------------------------
21.9 M    Trainable 

Sanity Checking: 0it [00:00, ?it/s]

Training: 493it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 18, global step 14606: 'val_loss' reached 0.10724 (best 0.10724), saving model to '/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 22, global step 16578: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 23, global step 17071: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 24, global step 17564: 'val_loss' was not in top 1


In [ ]:
n_epochs = 40
train_batch_size = 16
test_batch_size = 64
max_token_len = 384

data_module = RegulationDatasetModule(train_df, val_df, tokenizer, train_batch_size, test_batch_size ,max_token_len)
data_module.setup()

In [ ]:
steps_per_epoch=len(train_df) // train_batch_size
total_training_steps = steps_per_epoch * n_epochs

warmup_steps = total_training_steps // 3

In [ ]:
continuetrainer3 = pl.Trainer(max_epochs=40,
                             gpus=1,
                             logger=logger,
                             callbacks=[checkpoint_callback,early_stopping_callback],
                             resume_from_checkpoint="/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt")

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:54: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v2.0. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  "Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU 

In [ ]:
best_model3 = RegulationTagger.load_from_checkpoint(
    '/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt',
    n_classes=50,
    dropout_ratio=0.5,
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps,
    lr=0.0001
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
unfreeze_layers = ['layer.6','layer.7','bert.pooler', 'classifier.weight', 'classifier.bias']#'layer.10','layer.11',
for name ,param in best_model3.named_parameters():
    param.requires_grad = False
    for ele in unfreeze_layers:
        if ele in name:
            param.requires_grad = True
            break

In [ ]:
continuetrainer3.fit(best_model3, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1871: LightningDeprecationWarning: `trainer.resume_from_checkpoint` is deprecated in v1.5 and will be removed in v2.0. Specify the fit checkpoint path with `trainer.fit(ckpt_path=)` instead.
  stacklevel=5,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:606: UserWarning: Checkpoint directory /content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will

Sanity Checking: 0it [00:00, ?it/s]

Training: 493it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 22, global step 16578: 'val_loss' reached 0.10705 (best 0.10705), saving model to '/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 23, global step 17071: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 24, global step 17564: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 25, global step 18057: 'val_loss' was not in top 1


In [ ]:
n_epochs = 40
train_batch_size = 32
test_batch_size = 64
max_token_len = 384

data_module = RegulationDatasetModule(train_df, val_df, tokenizer, train_batch_size, test_batch_size ,max_token_len)
data_module.setup()

In [ ]:
steps_per_epoch=len(train_df) // train_batch_size
total_training_steps = steps_per_epoch * n_epochs

warmup_steps = total_training_steps // 3

In [ ]:
continuetrainer4 = pl.Trainer(max_epochs=40,
                             gpus=1,
                             logger=logger,
                             callbacks=[checkpoint_callback,early_stopping_callback],
                             resume_from_checkpoint="/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt")

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:54: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v2.0. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  "Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU 

In [ ]:
best_model4 = RegulationTagger.load_from_checkpoint(
    '/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt',
    n_classes=50,
    dropout_ratio=0.5,
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps,
    lr=0.00001
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
unfreeze_layers = ['layer.8','layer.9','bert.pooler', 'classifier.weight', 'classifier.bias']#'layer.10','layer.11',
for name ,param in best_model4.named_parameters():
    param.requires_grad = False
    for ele in unfreeze_layers:
        if ele in name:
            param.requires_grad = True
            break

In [ ]:
continuetrainer4.fit(best_model4, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1871: LightningDeprecationWarning: `trainer.resume_from_checkpoint` is deprecated in v1.5 and will be removed in v2.0. Specify the fit checkpoint path with `trainer.fit(ckpt_path=)` instead.
  stacklevel=5,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:606: UserWarning: Checkpoint directory /content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will

Sanity Checking: 0it [00:00, ?it/s]

Training: 493it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 23, global step 16825: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 24, global step 17072: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 25, global step 17319: 'val_loss' was not in top 1


In [ ]:
n_epochs = 40
train_batch_size = 4
test_batch_size = 64
max_token_len = 384

data_module = RegulationDatasetModule(train_df, val_df, tokenizer, train_batch_size, test_batch_size ,max_token_len)
data_module.setup()

In [ ]:
steps_per_epoch=len(train_df) // train_batch_size
total_training_steps = steps_per_epoch * n_epochs

warmup_steps = total_training_steps // 3

In [ ]:
continuetrainer5 = pl.Trainer(max_epochs=40,
                             gpus=1,
                             logger=logger,
                             callbacks=[checkpoint_callback,early_stopping_callback],
                             resume_from_checkpoint="/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt")

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:54: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v2.0. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  "Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU 

In [ ]:
best_model5 = RegulationTagger.load_from_checkpoint(
    '/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt',
    n_classes=50,
    dropout_ratio=0.5,
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps,
    lr=0.00005
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
unfreeze_layers = ['layer.0','bert.pooler', 'classifier.weight', 'classifier.bias']#'layer.10','layer.11',
for name ,param in best_model5.named_parameters():
    param.requires_grad = False
    for ele in unfreeze_layers:
        if ele in name:
            param.requires_grad = True
            break

In [ ]:
continuetrainer5.fit(best_model5, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:606: UserWarning: Checkpoint directory /content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type       | Params
------------------------------------------
0 | bert       | BertModel  | 108 M 
1 | classifier | Sequential | 38.5 K
2 | criterion  | BCELoss    | 0     
------------------------------------------
7.7 M     Trainable 

Sanity Checking: 0it [00:00, ?it/s]

Training: 283it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 25, global step 22777: 'val_loss' reached 0.10703 (best 0.10703), saving model to '/content/drive/MyDrive/DataSolve/competition/Bert+PytorchLightning/checkpoints/best-checkpoint.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 26, global step 24749: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 27, global step 26721: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 28, global step 28693: 'val_loss' was not in top 1


In [ ]:
best_model5.freeze()
best_model5.eval()

In [ ]:
def classify(context, model, tokenizer, label_names, threshold=0.5):
    
    encoding = tokenizer.encode_plus(
        context,
        add_special_tokens=True,
        max_length=384,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    _, prediction = model(
    encoding['input_ids'],
    encoding['attention_mask']
    )

    prediction = pd.DataFrame(prediction.flatten().numpy()).applymap(lambda y: 1 if y > threshold else 0).values.flatten()

    return prediction

In [ ]:
classify(test_df['context'][416], trained_model, tokenizer, label_cols)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [ ]:
result_table = test_df['context'].apply(lambda context: classify(context, best_model5, tokenizer, label_cols))

In [ ]:
result_table.apply(lambda x: x.tolist()).values.tolist()

In [ ]:
test_pred = pd.DataFrame(result_table.apply(lambda x: x.tolist()).values.tolist())
test_pred 


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4988,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4989,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4991,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
test_pred.columns = label_cols
test_pred = test_pred.astype('int')
test_final = pd.concat([test_df,test_pred], axis=1)

In [ ]:
id_table = []
for i in test_final['id']:
    for target in label_cols:
        id_table.append(str(i)+'_'+target)
        
prediction_table = []
for col, row in test_final.iterrows():
    for target in label_cols:
        prediction_table.append(row[target])

In [ ]:
answer = pd.concat([pd.DataFrame(id_table), pd.DataFrame(prediction_table)], axis =1)
answer.columns = ['id', 'predictions']
answer.to_csv('answer_t10.csv', header=True, index=False)

In [ ]:
answer

,id,predictions
0,4771_Accounting and Finance,1
1,4771_Antitrust,0
2,4771_Banking,0
3,4771_Broker Dealer,0
4,4771_Commodities Trading,0
...,...,...
249645,57235_Securities Management,0
249646,57235_Securities Sales,0
249647,57235_Securities Settlement,0
249648,57235_Trade Pricing,0
